### FaultFree_Training y FaultFree_Testing

In [1]:
import pyreadr
import numpy as np
import os
from rpy2.robjects import r, FloatVector, ListVector
import rpy2.robjects as ro

# -----------------------------
# Funciones
# -----------------------------

def load_rdata(path):
    """Carga un RData y devuelve el primer objeto dentro"""
    result = pyreadr.read_r(path)
    key = list(result.keys())[0]
    return result[key]

def dataframe_to_dict(df):
    """Convierte DataFrame Faulty a diccionario {fallo: [sim1, sim2, ...]}"""
    fault_dict = {}
    for col in df.columns:
        sims_col = df[col].values.tolist()
        fault_dict[col] = sims_col
    return fault_dict

def reduce_list_generic(sims, n=20):
    """Reduce a n simulaciones"""
    # Convertir DataFrame a lista de filas si es necesario
    if hasattr(sims, "iloc"):
        sims = sims.values.tolist()
    if len(sims) <= n:
        return sims
    idx = np.random.choice(len(sims), size=n, replace=False)
    return [sims[i] for i in idx]

def reduce_faulty_dict(faulty_dict, n=20):
    """Reduce cada fallo dentro del diccionario a n simulaciones"""
    reduced = {}
    for fault_name, sims in faulty_dict.items():
        reduced[fault_name] = reduce_list_generic(sims, n)
    return reduced

def pyobj_to_r(obj):
    """Convierte un objeto Python (lista o dict) a R compatible con rpy2"""
    if isinstance(obj, dict):
        r_dict = {}
        for k, v in obj.items():
            if isinstance(v, list):
                if all(isinstance(x, list) for x in v):
                    r_dict[k] = ListVector({str(i+1): FloatVector(x) for i, x in enumerate(v)})
                else:
                    r_dict[k] = FloatVector(v)
            else:
                r_dict[k] = v
        return ListVector(r_dict)
    elif isinstance(obj, list):
        if all(isinstance(x, list) for x in obj):
            return ListVector({str(i+1): FloatVector(x) for i, x in enumerate(obj)})
        else:
            return FloatVector(obj)
    else:
        raise TypeError(f"Tipo no soportado: {type(obj)}")

# -----------------------------
# Directorio de archivos
# -----------------------------
BASE_DIR = os.getcwd()

files = {
    "FaultFree_Training":  os.path.join(BASE_DIR, "TEP_FaultFree_Training.RData"),
    "FaultFree_Testing":   os.path.join(BASE_DIR, "TEP_FaultFree_Testing.RData"),
}

DEST_DIR = os.path.join(BASE_DIR, "DatasetReducido")
os.makedirs(DEST_DIR, exist_ok=True)

N = 20  # simulaciones por fallo

# -----------------------------
# Procesamiento y guardado
# -----------------------------
for name, path in files.items():
    print(f"\n=== Procesando {name} ===")
    data = load_rdata(path)

    # Convertir Faulty DataFrame a dict
    if "Faulty" in name and hasattr(data, "columns"):
        data = dataframe_to_dict(data)

    # Reducir simulaciones
    if "FaultFree" in name:
        reduced_data = reduce_list_generic(data, N)
    else:
        reduced_data = reduce_faulty_dict(data, N)

    # Convertir a R
    r_obj = pyobj_to_r(reduced_data)

    # Asignar al entorno global de R
    ro.globalenv[name] = r_obj

    # Guardar directamente como RData (rutas seguras para Windows)
    out_file = os.path.join(DEST_DIR, f"{name}_reduced.RData")
    out_file_r = out_file.replace("\\", "/")
    r(f'save(list="{name}", file="{out_file_r}")')
    print(f"{name}_reduced.RData guardado en {out_file_r}")

print("\nTodos los datasets reducidos guardados directamente en formato RData.")

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.



=== Procesando FaultFree_Training ===
FaultFree_Training_reduced.RData guardado en C:/Users/Alba/Documents/proyecto_analitica/DatasetReducido/FaultFree_Training_reduced.RData

=== Procesando FaultFree_Testing ===
FaultFree_Testing_reduced.RData guardado en C:/Users/Alba/Documents/proyecto_analitica/DatasetReducido/FaultFree_Testing_reduced.RData

Todos los datasets reducidos guardados directamente en formato RData.


### Faulty_Training y Faulty_Testing

In [2]:
import pyreadr
import numpy as np
import os
from rpy2.robjects import r, FloatVector, ListVector
import rpy2.robjects as ro

# -----------------------------
# Funciones
# -----------------------------

def load_rdata(path):
    """Carga un RData y devuelve el primer objeto dentro"""
    result = pyreadr.read_r(path)
    key = list(result.keys())[0]
    return result[key]

def dataframe_to_dict(df):
    """Convierte DataFrame Faulty a diccionario {fallo: [sim1, sim2, ...]}"""
    fault_dict = {}
    for col in df.columns:
        fault_dict[col] = df[col].values.tolist()
    return fault_dict

def reduce_list_generic(sims, n=20):
    """Reduce a n simulaciones"""
    if hasattr(sims, "iloc"):
        sims = sims.values.tolist()
    if len(sims) <= n:
        return sims
    idx = np.random.choice(len(sims), size=n, replace=False)
    return [sims[i] for i in idx]

def reduce_faulty_dict(faulty_dict, n=20):
    """Reduce cada fallo dentro del diccionario a n simulaciones"""
    reduced = {}
    for fault_name, sims in faulty_dict.items():
        reduced[fault_name] = reduce_list_generic(sims, n)
    return reduced

def pyobj_to_r(obj):
    """Convierte un objeto Python (lista o dict) a R compatible con rpy2"""
    if isinstance(obj, dict):
        r_dict = {}
        for k, v in obj.items():
            if isinstance(v, list):
                if all(isinstance(x, list) for x in v):
                    r_dict[k] = ListVector({str(i+1): FloatVector(x) for i, x in enumerate(v)})
                else:
                    r_dict[k] = FloatVector(v)
            else:
                r_dict[k] = v
        return ListVector(r_dict)
    elif isinstance(obj, list):
        if all(isinstance(x, list) for x in obj):
            return ListVector({str(i+1): FloatVector(x) for i, x in enumerate(obj)})
        else:
            return FloatVector(obj)
    else:
        raise TypeError(f"Tipo no soportado: {type(obj)}")

# -----------------------------
# Directorio de archivos (relativo)
# -----------------------------
BASE_DIR = os.getcwd()  # carpeta donde está el notebook

files = {
    "Faulty_Training": os.path.join(BASE_DIR, "TEP_Faulty_Training.RData"),
    "Faulty_Testing":  os.path.join(BASE_DIR, "TEP_Faulty_Testing.RData"),
}

# Carpeta para guardar datasets reducidos
DEST_DIR = os.path.join(BASE_DIR, "DatasetReducido")
os.makedirs(DEST_DIR, exist_ok=True)

N = 20  # simulaciones por fallo

# -----------------------------
# Procesamiento y guardado
# -----------------------------
for name, path in files.items():
    print(f"\n=== Procesando {name} ===")
    data = load_rdata(path)

    # Convertir Faulty DataFrame a dict
    if "Faulty" in name and hasattr(data, "columns"):
        data = dataframe_to_dict(data)

    # Reducir simulaciones
    if "FaultFree" in name:
        reduced_data = reduce_list_generic(data, N)
    else:
        reduced_data = reduce_faulty_dict(data, N)

    # Convertir a R
    r_obj = pyobj_to_r(reduced_data)

    # Asignar al entorno global de R
    ro.globalenv[name] = r_obj

    # Guardar directamente como RData (rutas seguras para Windows)
    out_file = os.path.join(DEST_DIR, f"{name}_reduced.RData")
    out_file_r = out_file.replace("\\", "/")
    r(f'save(list="{name}", file="{out_file_r}")')
    print(f"{name}_reduced.RData guardado en {out_file_r}")

print("\nTodos los datasets reducidos guardados directamente en formato RData.")


=== Procesando Faulty_Training ===
Faulty_Training_reduced.RData guardado en C:/Users/Alba/Documents/proyecto_analitica/DatasetReducido/Faulty_Training_reduced.RData

=== Procesando Faulty_Testing ===
Faulty_Testing_reduced.RData guardado en C:/Users/Alba/Documents/proyecto_analitica/DatasetReducido/Faulty_Testing_reduced.RData

Todos los datasets reducidos guardados directamente en formato RData.
